In [1]:
import pandas as pd
import time
import numpy as np
import requests
import re
from selenium import webdriver
from tqdm import tqdm
from lxml import html
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

### Пользовательские функции

In [2]:
# Функция extract_username принимает строку s и возвращает имя пользователя из URL-адреса,
# если он найден, или строку "null", если он не найден.
def extract_username(s):
    try:
        # Создаём регулярное выражение для поиска URL-адреса Instagram.
        pattern = r"instagram.com/(.*?)/"

        # Ищем URL-адрес в строке s.
        result = re.search(pattern, str(s))

        # Если URL-адрес найден, возвращаем имя пользователя.
        return result.group(1)

    # Если URL-адрес не найден, возвращаем строку "null".
    except:
        return "null"

In [3]:
def get_followers(nickname):
    # Отправляем GET-запрос на страницу пользователя
    url = f'https://www.pixwox.com/profile/{nickname}'

    # Заголовки запроса (например, User-Agent браузера Google Chrome)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    response = requests.get(url, headers=headers)

    # Если запрос прошел успешно, то парсим HTML
    if response.status_code == 200:
        tree = html.fromstring(response.content)

        # Ищем необходимый элемент по обновленному XPath
        element = tree.xpath('//div[contains(@class, "item_followers")]//div[contains(@class, "num")]')

        # Если элемент найден, то возвращаем значение его атрибута title
        if element:
            return int(element[0].get('title').replace(',', ''))
        else:
            return -1
    else:
        return -1

### Чтение данных и выделение ссылок на Instagram

In [4]:
# Читаем данные из файла main.xlsx в DataFrame.
df = pd.read_excel("baseline_dataset.xlsx")

# Читаем данные из файла final_full_followers_data.xlsx в DataFrame.
socials = pd.read_excel("final_full_followers_data.xlsx")

In [5]:
# Применяем функцию extract_username к столбцу "social_media" DataFrame.
df["username"] = df["social_media"].apply(extract_username)

# Получаем список уникальных значений столбца "username".
df["username"].unique()

array(['josko_gvardiol', 'nathanake', 'sergiogm_10', ..., 'cristianbro5',
       'berna94_', '2k.z4'], dtype=object)

In [6]:
# Объединяем DataFrame по столбцу "username".
merged = df.merge(socials, on="username", how="left")

# Заменяем значения -1 и None в столбце "followers" на 0.
merged["followers"] = merged["followers"].replace([-1, None], 0)

In [7]:
# Создаём список имён пользователей, у которых нет подписчиков.
problem_list = merged[merged["followers"] == 0]["username"].unique()

In [8]:
# Инициализируем словарь для хранения количества подписчиков для каждого имени пользователя.
followers_dict = {}

### Повторный парсинг ссылок для получения подписчиков

In [9]:
# Перебираем имена пользователей из списка problem_list.
for username in tqdm(problem_list):
    # Пытаемся получить количество подписчиков для данного имени пользователя.
    try:
        followers_dict[username] = get_followers(username)
    except:
        # Если имя пользователя не найдено, выводим сообщение и продолжаем цикл.
        print(f"{username} - not found")
        continue


100%|█████████████████████████████████████████| 283/283 [01:36<00:00,  2.94it/s]


In [10]:
# Создаём DataFrame из словаря followers_dict.
fixed = pd.DataFrame(
    list(followers_dict.items()), columns=["username", "new_followers"]
)

In [11]:
# Выбираем строки, где значение столбца "new_followers" больше 0, и выводим первые 10 строк результата.
fixed[fixed["new_followers"] > 0].head(10)

,username,new_followers
0,oscarbobb,305294
1,erling.haaland,38501935
2,levicolwill,565807
42,kastriotdermaku,10740
78,laparra17,28604
164,yans.yansane,11
184,noahsonkosundberg,12746
275,m_vucic_,1102


In [12]:
# Заменяем значение 0 в столбце "followers" на NaN.
merged["followers"] = merged["followers"].replace(0, np.nan)

### Обновление кол-ва подписчиков в старой таблице

In [13]:
# Объединяем DataFrame merged и fixed по столбцу "username" с помощью метода merge() и сохраняем результат в переменную merged.
merged = merged.merge(fixed, on="username", how="left")

# Заполняем значения NaN в столбце "new_followers" значением 0.
merged["new_followers"] = merged["new_followers"].fillna(0)

# Если значение в столбце "followers" равно NaN, заменяем его на значение из столбца "new_followers".
merged["followers"] = merged["followers"].fillna(merged["new_followers"])

# Удаляем столбцы "username" и "new_followers" из DataFrame merged.
merged = merged.drop(["username", "new_followers"], axis=1)

In [14]:
# Выводим первые строки DataFrame merged.
merged.head()

,full_name,age,place_of_birth,height,citizenship,position,player_agent,current_club,joined,outfitter,...,trophies_average_score,trophies_max_score,trophies_min_score,trophies_amount,club_trophies_total_score,club_trophies_average_score,club_trophies_max_score,club_trophies_min_score,club_trophies_amount,followers
0,Joško Gvardiol,22.0,Zagreb,1.85,Croatia,Defender - Centre-Back,1,Manchester City,7,0,...,6.814815,10,4,27,303,37.875,81,8,9,1199636.0
1,Nathan Benjamin Aké,29.0,Den Haag,1.80,Netherlands Cote d'Ivoire,Defender - Centre-Back,1,Manchester City,43,1,...,6.814815,10,4,27,303,37.875,81,8,9,2227475.0
2,Sergio Gómez Martín,23.0,Badalona,1.71,Spain,Defender - Left-Back,1,Manchester City,18,0,...,6.814815,10,4,27,303,37.875,81,8,9,475430.0
3,Kyle Andrew Walker,33.0,Sheffield,1.78,England,Defender - Right-Back,1,Manchester City,79,1,...,6.814815,10,4,27,303,37.875,81,8,9,4911531.0
4,Matheus Luiz Nunes,25.0,Rio de Janeiro,1.83,Portugal Brazil,midfield - Central Midfield,1,Manchester City,6,0,...,6.814815,10,4,27,303,37.875,81,8,9,447746.0


In [15]:
# Заменяем значения -1 и None в столбце "followers" на 0.
merged["followers"] = merged["followers"].replace([-1, None], 0)

### Сохранение датасета

In [16]:
# Сохраняем DataFrame merged в файл dataset_with_followers.xlsx без указания индекса.
merged.to_excel("dataset_with_followers.xlsx", index=False)